In [1]:
import numpy as np
from numba import cuda

# cuda.detect()


@cuda.jit
def another_kernel(a):
    """Commands to get thread positions"""
    # Get the thread position in a thread block
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    tz = cuda.threadIdx.z
    print(tx, ty, tz)

    # Get the id of the thread block
    block_x = cuda.blockIdx.x
    block_y = cuda.blockIdx.y
    block_z = cuda.blockIdx.z

    # Number of threads per block
    dim_x = cuda.blockDim.x
    dim_y = cuda.blockDim.y
    dim_z = cuda.blockDim.z

    # Global thread position
    pos_x = tx + block_x * dim_x
    pos_y = ty + block_y * dim_y
    pos_z = tz + block_z * dim_z

    # We can also use the grid function to get
    # the global position
    (pos_x, pos_y, pos_z) = cuda.grid(3)
    print(pos_x, pos_y, pos_z)
    a[:]=(tx,ty,tz)
    # For a 1-or 2-d grid use grid(1) or grid(2)
    # to return a scalar or a two tuple.


threadsperblock = (16, 16, 1)  # Should be a multiple of 32 if possible.
blockspergrid = (8, 8, 8)  # Blocks per grid
A=np.zeros(3)

another_kernel[blockspergrid, threadsperblock](A)

CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:

In [30]:
import numpy as np
from scipy.sparse import coo_matrix

def discretise_poisson(N):
    """Generate the matrix and rhs associated with the discrete Poisson operator."""

    nelements = 5 * N**2 - 16 * N + 16

    row_ind = np.empty(nelements, dtype=np.float64)
    col_ind = np.empty(nelements, dtype=np.float64)
    data = np.empty(nelements, dtype=np.float64)

    f = np.empty(N * N, dtype=np.float64)

    count = 0
    for j in range(N):
        for i in range(N):
            if i == 0 or i == N - 1 or j == 0 or j == N - 1:
                row_ind[count] = col_ind[count] = j * N + i
                data[count] =  1
                f[j * N + i] = 0
                count += 1

            else:
                row_ind[count : count + 5] = j * N + i
                col_ind[count] = j * N + i
                col_ind[count + 1] = j * N + i + 1
                col_ind[count + 2] = j * N + i - 1
                col_ind[count + 3] = (j + 1) * N + i
                col_ind[count + 4] = (j - 1) * N + i

                data[count] = 4 * (N - 1)**2
                data[count + 1 : count + 5] = - (N - 1)**2
                f[j * N + i] = 1

                count += 5
    print(row_ind,'\n')
    print(col_ind,'\n')
    print(data)
    return coo_matrix((data, (row_ind, col_ind)), shape=(N**2, N**2)).tocsr(), f

[ 0.  1.  2.  3.  4.  5.  5.  5.  5.  5.  6.  6.  6.  6.  6.  7.  8.  9.
  9.  9.  9.  9. 10. 10. 10. 10. 10. 11. 12. 13. 14. 15.] 

[ 0.  1.  2.  3.  4.  5.  6.  4.  9.  1.  6.  7.  5. 10.  2.  7.  8.  9.
 10.  8. 13.  5. 10. 11.  9. 14.  6. 11. 12. 13. 14. 15.] 

[ 1.  1.  1.  1.  1. 36. -9. -9. -9. -9. 36. -9. -9. -9. -9.  1.  1. 36.
 -9. -9. -9. -9. 36. -9. -9. -9. -9.  1.  1.  1.  1.  1.]


In [32]:
A,f=discretise_poisson(4)
print(A.toarray(),A.shape)
print(f)

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -9.  0.  0. -9. 36. -9.  0.  0. -9.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. -9.  0.  0. -9. 36. -9.  0.  0. -9.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -9.  0.  0. -9. 36. -9.  0.  0. -9.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -9.  0.  0. -9. 36. -9.  0.  0. -9.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. 